# Character-Level LSTM in PyTorch

In this notebook, I'll construct a character-level LSTM with PyTorch. The network will train character by character on some text, then generate new text character by character. As an example, I will train on Anna Karenina. **This model will be able to generate new text based on the text from the book!**

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

First let's load in our required resources for data loading and model creation.

In [1]:
import numpy as np
from numpy import random
import torch
from torch import nn
import torch.nn.functional as F

## This line of code ensures the reproducibility of the code
torch.manual_seed(42)

## Load in Data

Then, we'll load the Anna Karenina text file and convert it into integers for our network to use. 

In [2]:
# open text file and read in data as `text`
with open('data/anna.txt', 'r') as f:
    text = f.read()

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [3]:
display(text[:100])
print(f'\n\n{text[:100]}')

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'



Chapter 1


Happy families are all alike; every unhappy family is unhappy in its own
way.

Everythin


### Tokenization

In the cells, below, I'm creating a couple **dictionaries** to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

we have 83 characters in total and we will give them numbers from 0 to 82

In [4]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
# We use sorted to make sure the code is reproducible
chars = sorted(set(text))
int2char = dict(enumerate(chars))
char2int = {ch:ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

And we can see those same characters from above, encoded as integers.

In [5]:
encoded[:100]

array([31, 64, 57, 72, 76, 61, 74,  1, 16,  0,  0,  0, 36, 57, 72, 72, 81,
        1, 62, 57, 69, 65, 68, 65, 61, 75,  1, 57, 74, 61,  1, 57, 68, 68,
        1, 57, 68, 65, 67, 61, 26,  1, 61, 78, 61, 74, 81,  1, 77, 70, 64,
       57, 72, 72, 81,  1, 62, 57, 69, 65, 68, 81,  1, 65, 75,  1, 77, 70,
       64, 57, 72, 72, 81,  1, 65, 70,  1, 65, 76, 75,  1, 71, 79, 70,  0,
       79, 57, 81, 13,  0,  0, 33, 78, 61, 74, 81, 76, 64, 65, 70])

## Pre-processing the data

As you can see in our char-RNN image above, our LSTM expects an input that is **one-hot encoded** meaning that each character is converted into an integer (via our created dictionary) and *then* converted into a column vector where only it's corresponding integer index will have the value of 1 and the rest of the vector will be filled with 0's. Since we're one-hot encoding the data, let's make a function to do that!


In [6]:
## That was my initial approach which gave an error during model training
def one_hot_encode(arr, vector_length):
    '''
        Note: The function will through an error if one of the elements of the array
        is of a value equal to or greater than the vector_length
    '''
    
    # Initialize the the encoded array
    ## Notice the float32 since it's PyTorch's default dtype for the weights
    one_hot = np.zeros((arr.size, vector_length), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(arr.size), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape(*arr.shape, vector_length)
    
    return one_hot

### explaining the code above
- we create an array of a certain shape with all its values = 0
- we change the values according to the position of the integer values of the characters to 1s like the code below

In [7]:
## helper code to illustrate the cell above
## here I am creating a numpy array of 6 Rows X 4 Columns
some_array = np.arange(1, 25, dtype=np.float32).reshape(6,4)

## checking the created array
print(f'The full array is:\n{some_array}\n')

## Now I will be choosing certain elements of the array by passing 2 lists as indices
## The first list is the indices to the rows I am slicing
## The second list is the indices to the columns within those rows
## Now I will be extracting the 1, the 7 and the 12
print(f'\nExtracting specific elements from the array:\n{some_array[[0,1,2], [0,2,-1]]}')

## Now I will set the values above to equal 100 and print the array after this modification
some_array[[0,1,2], [0,2,-1]] = 100
print(f'\nThe array after modification is:\n{some_array}')

The full array is:
[[ 1.  2.  3.  4.]
 [ 5.  6.  7.  8.]
 [ 9. 10. 11. 12.]
 [13. 14. 15. 16.]
 [17. 18. 19. 20.]
 [21. 22. 23. 24.]]


Extracting specific elements from the array:
[ 1.  7. 12.]

The array after modification is:
[[100.   2.   3.   4.]
 [  5.   6. 100.   8.]
 [  9.  10.  11. 100.]
 [ 13.  14.  15.  16.]
 [ 17.  18.  19.  20.]
 [ 21.  22.  23.  24.]]


In [8]:
# check that the function works as expected
test_seq = np.array([[[[3, 5, 7]]]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)
print('\n' + '='*50 + '\n')

test_seq = np.array([[3, 5, 7], [1, 1, 2]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[[[0. 0. 0. 1. 0. 0. 0. 0.]
    [0. 0. 0. 0. 0. 1. 0. 0.]
    [0. 0. 0. 0. 0. 0. 0. 1.]]]]]


[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1.]]

 [[0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0.]]]


We can see our `one_hot_encode` function is returing the predicted output with the proper dimensions

---
## Making training mini-batches


To train on this data, we also want to create mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>

In this example, we'll take the encoded characters (passed in as the `arr` parameter) and split them into multiple sequences, given by `batch_size`. Each of our sequences will be `seq_length` long.

### Creating Batches

**1. The first thing we need to do is discard some of the text so we only have completely full mini-batches. **

Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences in a batch) and $M$ is the seq_length or number of time steps in a sequence. Then, to get the total number of batches, $K$, that we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

**2. After that, we need to split `arr` into $N$ batches. ** 

You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences in a batch, so let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

**3. Now that we have this array, we can iterate through it to get our mini-batches. **

The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `seq_length`. We also want to create both the input and target arrays. Remember that the targets are just the inputs shifted over by one character. The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of tokens in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `seq_length` wide.

> **TODO:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

#### hints:
- **N** = batch size = the number of sequences in the batch
- **M** = the full sequence length
- **K** = the number of batches

after understanding the steps above, we will split our data into a number of batches, these batches will be an integer and any remaining characters will be dropped, so once we have **N** batches in total, we will feed them to the model with **sequence length** a time, that is not the full sequence length which is the total number of characters at each batch, it's just the number of characters that are fed to the network per time, so the window for our characters will be of **number of rows = batch size** and **number of columns = sequence length**

- each sequence will be fed to the model 1 mini sequence length a time
- the batch size is not the same as the number of batches

In [9]:
def get_batches(arr, batch_size, seq_length, print_n_seq=False):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    seq_window_size = batch_size * seq_length
    
    ## TODO: Get the number of batched sequences we can make
    n_seq = int(arr.size / seq_window_size)
    
    if print_n_seq:
        print(f'The inputs will result in {n_seq} sequences')
    
    ## TODO: Keep only enough characters to make full batches
    arr = arr[:n_seq * seq_window_size]
    
    ## TODO: Reshape into batch_size rows
    ## This returns a 2d Array
    arr = arr.reshape(batch_size, -1)
    
    ## TODO: Iterate over the batches using a window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        y[:, :-1] = x[:, 1:]
        try:
            y[:, -1] = arr[:, n+seq_length]
        except IndexError:
            y[:, -1] = arr[:, 0]
        yield x, y

### Test Your Implementation

Now I'll make some data sets and we can check out what's going on as we batch data. Here, as an example, I'm going to use a batch size of 8 and 50 sequence steps.

In [10]:
batches = get_batches(encoded, 8, 50, True)
x, y = next(batches)

The inputs will result in 4963 sequences


In [11]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[31 64 57 72 76 61 74  1 16  0]
 [75 71 70  1 76 64 57 76  1 57]
 [61 70 60  1 71 74  1 57  1 62]
 [75  1 76 64 61  1 59 64 65 61]
 [ 1 75 57 79  1 64 61 74  1 76]
 [59 77 75 75 65 71 70  1 57 70]
 [ 1 29 70 70 57  1 64 57 60  1]
 [43 58 68 71 70 75 67 81 13  1]]

y
 [[64 57 72 76 61 74  1 16  0  0]
 [71 70  1 76 64 57 76  1 57 76]
 [70 60  1 71 74  1 57  1 62 71]
 [ 1 76 64 61  1 59 64 65 61 62]
 [75 57 79  1 64 61 74  1 76 61]
 [77 75 75 65 71 70  1 57 70 60]
 [29 70 70 57  1 64 57 60  1 75]
 [58 68 71 70 75 67 81 13  1  3]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[25  8 60 11 45 27 28 73  1  2]
 [17  7 20 73 45  8 60 45 73 60]
 [27 20 80 73  7 28 73 60 73 65]
 [17 73 45  8 27 73 66  8 46 27]
 [73 17 60 12 73  8 27 28 73 45]
 [66 64 17 17 46  7 20 73 60 20]
 [73 76 20 20 60 73  8 60 80 73]
 [47 35 43  7 20 17 24 50 37 73]]

y
 [[ 8 60 11 45 27 28 73  1  2  2]
 [ 7 20 73 45  8 60 45 73 60 45]
 [20 80 73  7 28 73 60 73 65  7]
 [73 45  8 27 73 66  8 46 27 65]
 [17 60 12 73  8 27 28 73 45 27]
 [64 17 17 46  7 20 73 60 20 80]
 [76 20 20 60 73  8 60 80 73 17]
 [35 43  7 20 17 24 50 37 73 36]]
 ```
 although the exact numbers may be different. Check to make sure the data is shifted over one step for `y`.

In [12]:
## checking if the above encoded values make sense
for seq in x:
    print(''.join([int2char[char] for char in seq]))
    print('='*60)

Chapter 1


Happy families are all alike; every un
son that attracted her attention was her husband. 
end or a foe, he avoided his father. He
looked rou
s the chief though hidden
interest of his life, of
 saw her tear-stained, pitiful, sweet face,
misera
cussion and analysis, was in principle disagreeabl
 Anna had said that Dolly would excuse it. And thi
Oblonsky. "But _they_ cannot grasp that,
_they_ ar


In [13]:
## checking the last sequence in the batches
for seq in range(4963-1):
    x, y = next(batches)

for seq in x:
    print(''.join([int2char[char] for char in seq]))
    print('='*60)

s the train stopped and she got out, the first
per
ion had occurred to
him whether Vronsky were a fri
service
or not brought him to the other and perhap
r house before the theater,
went into her room and
erpretations, just because it paved the way
to dis
hamber Vronsky had suggested, but the
one of which
 equally," he said,
looking over his pince-nez at 
, and how to
subscribe to our email newsletter to 


In [14]:
for seq in y:
    print(''.join([int2char[char] for char in seq]))
    print('='*60)

 the train stopped and she got out, the first
perC
on had occurred to
him whether Vronsky were a fris
ervice
or not brought him to the other and perhape
 house before the theater,
went into her room ands
rpretations, just because it paved the way
to dis 
amber Vronsky had suggested, but the
one of whichc
equally," he said,
looking over his pince-nez at  
 and how to
subscribe to our email newsletter to O


In [15]:
## Modifying the function above regarding the last character of the last batch of sequences

def get_batches(arr, batch_size, seq_length, print_n_seq=False):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    seq_window_size = batch_size * seq_length
    
    ## TODO: Get the number of batched sequences we can make
    n_seq = int(arr.size / seq_window_size)
    
    if print_n_seq:
        print(f'The inputs will result in {n_seq} sequences')
    
    ## TODO: Keep only enough characters to make full batches
    arr = arr[:n_seq * seq_window_size]
    
    ## TODO: Reshape into batch_size rows
    ## This returns a 2d Array
    arr = arr.reshape(batch_size, -1)
    
    ## TODO: Iterate over the batches using a window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        y[:, :-1] = x[:, 1:]
        try:
            y[:, -1] = arr[:, n+seq_length]
        except IndexError:
            y[:-1, -1] = arr[1:, 0]
            y[-1, -1] = arr[0, 0]
        yield x, y

In [16]:
batches = get_batches(encoded, 8, 50, True)
x, y = next(batches)

for seq in x:
    print(''.join([int2char[char] for char in seq]))
    print('='*60)

The inputs will result in 4963 sequences
Chapter 1


Happy families are all alike; every un
son that attracted her attention was her husband. 
end or a foe, he avoided his father. He
looked rou
s the chief though hidden
interest of his life, of
 saw her tear-stained, pitiful, sweet face,
misera
cussion and analysis, was in principle disagreeabl
 Anna had said that Dolly would excuse it. And thi
Oblonsky. "But _they_ cannot grasp that,
_they_ ar


In [17]:
## checking the last sequence in the batches

for seq in range(4963-1):
    x, y = next(batches)

for seq in y:
    print(''.join([int2char[char] for char in seq]))
    print('='*60)

 the train stopped and she got out, the first
pers
on had occurred to
him whether Vronsky were a frie
ervice
or not brought him to the other and perhaps
 house before the theater,
went into her room and 
rpretations, just because it paved the way
to disc
amber Vronsky had suggested, but the
one of which 
equally," he said,
looking over his pince-nez at O
 and how to
subscribe to our email newsletter to C


---
## Defining the network with PyTorch

Below is where you'll define the network.

<img src="assets/charRNN.png" width=500px>

Next, you'll use PyTorch to define the architecture of the network. We start by defining the layers and operations we want. Then, define a method for the forward pass. You've also been given a method for predicting characters.

### Model Structure

In `__init__` the suggested structure is as follows:
* Create and store the necessary dictionaries (this has been done for you)
* Define an LSTM layer that takes as params: an input size (the number of characters), a hidden layer size `n_hidden`, a number of layers `n_layers`, a dropout probability `drop_prob`, and a batch_first boolean (True, since we are batching)
* Define a dropout layer with `drop_prob`
* Define a fully-connected layer with params: input size `n_hidden` and output size (the number of characters)
* Finally, initialize the weights (again, this has been given)

Note that some parameters have been named and given in the `__init__` function, and we use them and store them by doing something like `self.drop_prob = drop_prob`.

---
### LSTM Inputs/Outputs

You can create a basic [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) as follows

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
```

where `input_size` is the number of characters this cell expects to see as sequential input, and `n_hidden` is the number of units in the hidden layers in the cell. And we can add dropout by adding a dropout parameter with a specified probability; this will automatically add dropout to the inputs or outputs. Finally, in the `forward` function, we can stack up the LSTM cells into layers using `.view`. With this, you pass in a list of cells and it will send the output of one cell into the next cell.

We also need to create an initial hidden state of all zeros. This is done like so

```python
self.init_hidden()
```

In [18]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [19]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                 drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch:ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the layers of the model
        
        ## define the lstm layer
        self.lstm = nn.LSTM(
            input_size=len(tokens), hidden_size =n_hidden,
            num_layers =n_layers, dropout=drop_prob, batch_first=True
        )
        ## explaining the variables above on creating the LSTM
        ### the first argument = input size
        ### We are still feeding the network one input value per time step
        ### However, the inputs are in the form of one hot encoded vectors
        ### of length = the number of characters and that's why our input
        ### size is not 1 like the RNN example but the number of the characters        
        
        ## define the dropout layer
        ### I am commenting the line below as I won't drop out between the
        ### LSTM output and the fully connected layer
        #self.dropout = nn.Dropout(p=drop_prob)
        
        ## define a fully connected layer
        self.fc = nn.Linear(n_hidden, len(tokens))
        ### The number of the outputs of the fully connected layer
        ### equals to the length of the one hot encoded vector with
        ### class scores for each possible character
    
    def forward(self, x, hidden):
        '''
        Forward pass through the network. 
        These inputs are x, and the hidden/cell state `hidden`.
        '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        ## r_output stands for the recurrent output
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        #out = self.dropout(r_output)
        
        ################################################################
        ## r_output has a shape of batch_size X seq length X n_hidden ##
        ################################################################
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = r_output.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (
                weight.new(
                    self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                weight.new(
                    self.n_layers, batch_size, self.n_hidden).zero_().cuda()
            )
        else:
            hidden = (
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_()
            )
        
        return hidden

## Time to train

The train function gives us the ability to set the number of epochs, the learning rate, and other parameters.

Below we're using an Adam optimizer and cross entropy loss since we are looking at character class scores as output. We calculate the loss and perform backpropagation, as usual!

A couple of details about training: 
>* Within the batch loop, we detach the hidden state from its history; this time setting it equal to a new *tuple* variable because an LSTM has a hidden state that is a tuple of the hidden and cell states.
* We use [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) to help prevent exploding gradients.

In [20]:
def train(net, data, epochs=10, batch_size=10, seq_length=50,
          lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' 
    Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per batch
        seq_length: Number of characters per mini-sequence
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    '''
    
    ## setting an initial validation loss for early stopping
    valid_loss_min = np.Inf
    
    ## putting the network on train mode
    net.train()
    
    ## moving the network to the GPU is a supported GPU is available
    if(train_on_gpu):
        net.cuda()
    
    ## setting the optimizer and the criterion
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    counter = 0
    early_stop_counter = 0
    n_chars = len(net.chars)
    
    for epoch in range(epochs):
        # initialize hidden state
        ## we can either use an if statement to initiate h once per training
        ## or we can remove the if statement and initiate h each epoch
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            ## moving the data to the GPU is a supported GPU is available
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])
            
            # zero accumulated gradients
            opt.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            ## the .long converts the dtype of the tensor to long
            ## since the nn.CrossEntropyLoss() expects dtype = torch.long
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                
                with torch.no_grad():
                    # Get validation loss
                    val_h = net.init_hidden(batch_size)
                    val_losses = []
                    net.eval()
                    for x, y in get_batches(val_data, batch_size, seq_length):
                        # One-hot encode our data and make them Torch tensors
                        x = one_hot_encode(x, n_chars)
                        x, y = torch.from_numpy(x), torch.from_numpy(y)

                        # Creating new variables for the hidden state, otherwise
                        # we'd backpropagate through the entire training history
                        val_h = tuple([each.data for each in val_h])

                        ## Moving the data to the GPU
                        inputs, targets = x, y
                        if(train_on_gpu):
                            inputs, targets = inputs.cuda(), targets.cuda()

                        output, val_h = net(inputs, val_h)
                        val_loss = criterion(output, targets.view(batch_size*seq_length).long())

                        val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print(f"Epoch: {epoch+1:2}/{epochs}...  "
                      f"Step: {counter:4}...  "
                      f"Loss: {loss.item():.4f}...  "
                      f"Val Loss: {np.mean(val_losses):.4f}")

        # save model if validation loss has decreased
        if val_loss <= valid_loss_min:
            print(f'\nValidation loss decreased ({valid_loss_min:.4f} --> {val_loss:.4f})'
                  '. Saving model...\n')
            best_model_params = net.state_dict()
            torch.save(best_model_params, './checkpoints/model_status.pt')
            valid_loss_min = val_loss
            early_stop_counter = 0
        else:
            early_stop_counter += 1
            print()
        
        ## Early stopping the training:
        if early_stop_counter == 5:
            print('The model is no longer improving so stopping the training process')
            break
    
    return best_model_params

## Instantiating the model

Now we can actually train the network. First we'll create the network itself, with some given hyperparameters. Then, define the mini-batches sizes, and start training!

In [21]:
## TODO: set your model hyperparameters
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


### Set your training hyperparameters!

In [22]:
%%time
batch_size = 256
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
best_model_params = train(
    net, encoded, epochs=n_epochs, batch_size=batch_size,
    seq_length=seq_length, lr=0.001, print_every=25)

Epoch:  1/20...  Step:   25...  Loss: 3.1128...  Val Loss: 3.1011
Epoch:  1/20...  Step:   50...  Loss: 3.1082...  Val Loss: 3.0957

Validation loss decreased (inf --> 3.0883). Saving model...

Epoch:  2/20...  Step:   75...  Loss: 3.1033...  Val Loss: 3.0924
Epoch:  2/20...  Step:  100...  Loss: 3.0995...  Val Loss: 3.0847
Epoch:  2/20...  Step:  125...  Loss: 3.0427...  Val Loss: 3.0202

Validation loss decreased (3.0883 --> 3.0139). Saving model...

Epoch:  3/20...  Step:  150...  Loss: 2.8439...  Val Loss: 2.8314
Epoch:  3/20...  Step:  175...  Loss: 2.6944...  Val Loss: 2.6522
Epoch:  3/20...  Step:  200...  Loss: 2.5317...  Val Loss: 2.4774

Validation loss decreased (3.0139 --> 2.4746). Saving model...

Epoch:  4/20...  Step:  225...  Loss: 2.4286...  Val Loss: 2.3753
Epoch:  4/20...  Step:  250...  Loss: 2.3602...  Val Loss: 2.3038
Epoch:  4/20...  Step:  275...  Loss: 2.2627...  Val Loss: 2.2253

Validation loss decreased (2.4746 --> 2.2273). Saving model...

Epoch:  5/20...  

## Getting the best model

To set your hyperparameters to get the best performance, you'll want to watch the training and validation losses. If your training loss is much lower than the validation loss, you're overfitting. Increase regularization (more dropout) or use a smaller network. If the training and validation losses are close, you're underfitting so you can increase the size of the network.

## Hyperparameters

Here are the hyperparameters for the network.

In defining the model:
* `n_hidden` - The number of units in the hidden layers.
* `n_layers` - Number of hidden LSTM layers to use.

We assume that dropout probability and learning rate will be kept at the default, in this example.

And in training:
* `batch_size` - Number of sequences running through the network in one pass.
* `seq_length` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lr` - Learning rate for training

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `n_hidden` and `n_layers`. I would advise that you always use `n_layers` of either 2 or 3. The `n_hidden` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `n_hidden` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

## Checkpoint

After training, we'll save the model so we can load it again later if we need too. Here I'm saving the parameters needed to create the same architecture, the hidden layer hyperparameters and the text characters.

In [23]:
# change the name, for saving multiple files
model_name = 'rnn_20_epoch_02072022.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': best_model_params,
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

---
## Making Predictions

Now that the model is trained, we'll want to sample from it and make predictions about next characters! To sample, we pass in a character and have the network predict the next character. Then we take that character, pass it back in, and get another predicted character. Just keep doing this and you'll generate a bunch of text!

### A note on the `predict`  function

The output of our RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

> To actually get the next character, we apply a softmax function, which gives us a *probability* distribution that we can then sample to predict the next character.

### Top K sampling

Our predictions come from a categorical probability distribution over all the possible characters. We can make the sample text and make it more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving us completely absurd characters while allowing it to introduce some noise and randomness into the sampled text. Read more about [topk, here](https://pytorch.org/docs/stable/torch.html#torch.topk).


In [24]:
def predict(net, char, h=None, top_k=1, debug=False):
    ''' 
    Given a character, predict the next character.
    Returns the predicted character and the hidden state.
        
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per batch
        seq_length: Number of characters per mini-sequence
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss

    '''
        
    ## converting the input from a string character to a 2d numpy array
    x = np.array([[net.char2int[char]]])
    if debug:
        print(f'The encoded character value is: {x}\n')
    
    ## converting the integer encoded value of the character to a one hot encoded vector
    ## this returns a 3d numpy array
    x = one_hot_encode(x, len(net.chars))
    if debug:
        print(f'The one hot encoded character value is:\n{x}\n')
    
    ## converting the one hot encoded vector to a tensor
    inputs = torch.from_numpy(x)

    ## moving the input data to the GPU if available
    if(train_on_gpu):
        inputs = inputs.cuda()

#     # detach hidden state from history
#     h = tuple([each.data for each in h])

    ## get the predicted next character and the new hidden state from the model
    out, h = net(inputs, h)
    if debug:
        print(f'The predicted outputs of the model are:\n{out}\n')
    
    ## get the character probabilities, we must use the .detach() method
    ## or else we won't be able to convert the output tensor to a numpy array
    p = F.softmax(out, dim=1).detach()
    if(train_on_gpu):
        p = p.cpu() # move to cpu

    ## get the top characters, p = the probabilities arranged from the highest to the lowest
    ## top_ch = the top characters from the most probable to the least probable
    p, top_ch = p.topk(top_k)
    top_ch = top_ch.numpy().squeeze()

    # select the likely next character with some element of randomness
    p = p.numpy().squeeze()

    if top_k == 1:
        ## returns the encoded integer value to the predicted character
        char = top_ch.item()
    else:
        char = np.random.choice(top_ch, p=p/p.sum())

    # return the encoded value of the predicted char and the hidden state
    return net.int2char[char], h

### Priming and generating text 

Typically you'll want to prime the network so you can build up a hidden state, this means you would like the network to look at a certain sequence of characters one character per time so that it builds up a memory connecting the characters forming that sequence. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [25]:
def sample(net, size, prime='The', top_k=1):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    ## convert the prime word to a list
    chars = list(prime)
    
    ## initiate a hidden state before feeding the network the first character
    ## in the prime word, we are specifying the batch_size as 1 here
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    ## Now pass in the previous character and get a new one
    ## Then append it to the chars list
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [26]:
print(sample(net, 1000, prime='Anna', top_k=5))

Anna...
What don't talk able that they meant with
her table, but there was, I'm set it.'
After the first
painty that have had tell him of it all the stats, and he was not, a money for her, but the came a crusid and close face out intimes of his hands. This
is. The stroight wo do it only
bust both anything. A certain of the
country, wonders on the forget, and at her sould for a more attacking a child of it is there was not formert in this about it on
the creater and streng, so the for that wo shall have heard in the care, and, and they was the
sour in the strange, what was the position on the stendly for mer them of this, and was in a sing than how she would not see her and seems in what I shall have no one and happiness would not looked
the
streng. It was so it were a listered from the make worred."

"You say? Why do you think to many to
go be doing ord it. And that is it that haste, as she said at that I'll say. By the stains that is the strange to be triff."

"In at the posing of an 

In [27]:
print(sample(net, 1000, prime='Anna', top_k=1))

Anna and the same time to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to h

## Loading a checkpoint

In [28]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_20_epoch_02072022.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [29]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin said, and she had
believed it, she was stricking high, the fartion was a little,
with his
ball of the strong of the coats and steaking or his brother than
this was this across, and settle and wine,
standing to
him, that he was not set her, with his which he was clossing to the pryass, and so those shake of
her son an old man when, too he went towards them with a
sight whee she was not all the brind and a gray at a caper, and the
peasants were concement.

"I'd got out of the same. Whan's it we should be so something. They're still wording a master, as I have not to
get to heve the paintal and fastant."

"We arm anything to be idle,"
he seemed to strake into his stiden. But his words, seeing her face was not between.

"Only more face," said Stepan Arkadyevitch."

"Yes, I can't going, but that's not times. I shall see him about at her. And he's a state of there may."

"Not it must take to have meeting about her he too," answered Levin. "How
can't be so much as she had
been all t

In [30]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=2, prime="And Levin said"))

And Levin said to him. He was already and he was something of the course, and he was not a long of her hands, and
she had not the sat of the same stream with her since.

"I don't know, and I wonder to see your latter, when I say, a mistake of
this an experiate that I was never for a like the same of the street and the
conversation of the same thing of the strenged of the conversation of the chance of the strange of the countess of the
country of the strange of her face on the station of the same time, who was struck at the carriage to her a smire of him, and to see them, and too was a smire of him, and the same time that the streem of the
conversation of him and he had not the province of his face was
standing to her.

"I don't know," said Stepan Arkadyevitch, and the state of the same state and straight of the convincess of the stair, which had been the person he had no deciet of his face, and she
would have
been a conscious the comparion of this conversation of the carriage to her a 

In [31]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=1, prime="And Levin said"))

And Levin said to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to him to hi

### Notes about this notebook:
1. As we can see from the predictions above, if the network shall return only the most probable character then it might get stuck in a loop of the same characters being predicted and fed to the network. So, changing k at the **top_k** to 2 or more will give us much better predictions.
2. We can see the model is giving logical predictions on setting **k to values between 2 to 5**, we managed to get complete words with the correct spelling, proper spacing and even some punctuation but the words themselves might not make much sense regarding the context.
3. Another **hyperparameter** that affects the training process and doesn't make much sense is the encoded numeric value for each character. An early data preparation step at this notebook was converting each character to a numeric value **"token"** and then converting these tokens to one-hot encoded vectors. Such an approach shouldn't be affected by the token value as eventually all the values will be converted to a **vector of 1s and 0s**. However, when I used the **sorted** function to ensure the reproducibility of my code, the model overfitted although all the other **hyperparameters** weren't changed and the model started predicting characters that make no sense at all and almost with no spacing.
4. To avoid the overfitting mentioned at the previous point, I reduced the number of **training epochs from 50 to 20**, the reason behind that is the log printed during training. As you can see, epoch 20 was the first epoch in which the training loss was less than the validation loss. So, we could have added another early stopping criteria to stop the training when both the training loss and validation loss are almost identical.